# The Very Hungry Snake Game

You know the classic Snake Game, right? On Day 20 and 21 I was tasked to recreate this iconic game using Python's turtle graphics. 

Requirements:
- Control a snake, make it eat food, and grow longer without hitting the walls or its own tail

I added:
- Countdown
- Move faster after each bite
- Replay option
- Food can't appear inside the snake

### Imported libraries

In [1]:
from turtle import Screen, Turtle

In [2]:
import random
import time

### Creating the Food Class

First, I needed the snake to have something to eat. So, I created a Food class to handle this. The food appears as a small red dot at random spots on the screen.

In [3]:
class Food(Turtle):
    def __init__(self, snake_segments):
        super().__init__()
        self.snake_segments = snake_segments
        self.shape("circle")
        self.penup()
        self.shapesize(stretch_len=0.5, stretch_wid=0.5)
        self.color("red")
        self.speed("fastest")
        self.refresh()
    
    def is_position_safe(self, x, y):
        """Check if the position (x, y) is not overlapping with any snake segment."""
        for segment in self.snake_segments:
            if segment.distance(x, y) < 15:
                return False
        return True

    def refresh(self):
        """Set food to a new position that's not overlapping with the snake."""
        safe = False
        while not safe:
            random_x = random.randint(-280, 280)
            random_y = random.randint(-280, 280)
            safe = self.is_position_safe(random_x, random_y)
        
        self.goto(random_x, random_y)

- The food class inherits from Turtle because Turtle has a lot of useful methods.
- The food shape is set to a small red circle.
- The refresh method randomly places the food on the screen. This way, every time the snake eats, new food appears in a different spot.
- The food can't appear within the snake's body.

### Building the scoreboard

Next, I needed to keep track of the score. So, I created a Scoreboard class. This class updates the score and displays it on the screen. It also shows a "GAME OVER" message when the game ends.

In [4]:
ALIGNMENT = "center"
FONT = ("Arial", 24, "normal")

class Scoreboard(Turtle):
    def __init__(self):
        super().__init__()
        self.score = 0
        self.color("white")
        self.penup()
        self.goto(0, 270)
        self.hideturtle()
        self.update_scoreboard()

    def update_scoreboard(self):
        self.write(f"Score: {self.score}", align=ALIGNMENT, font=FONT)

    def game_over(self):
        self.goto(0, 0)
        self.write("GAME OVER", align=ALIGNMENT, font=FONT)

    def increase_score(self):
        self.score += 1
        self.clear()
        self.update_scoreboard()
    
    def reset(self):
        self.score = 0
        self.clear()
        self.update_scoreboard()

- Scoreboard also extends Turtle and I set it up to display the score at the top of the screen.
- The update_scoreboard method shows the current score.
- The game_over method displays a "GAME OVER" message in the center of the screen.
- The increase_score method bumps up the score and refreshes the display.

### Creating the snake

Then, I worked on the snake itself. This part was the most fun! The Snake class manages the snake's segments, its movement, and handling collisions.

In [5]:
STARTING_POSITIONS = [(0, 0), (-20, 0), (-40, 0)]
MOVE_DISTANCE = 20
UP = 90
DOWN = 270
LEFT = 180
RIGHT = 0

class Snake:
    def __init__(self):
        self.segments = []
        self.create_snake()
        self.head = self.segments[0]
    
    def create_snake(self):
        for position in STARTING_POSITIONS:
            self.add_segment(position)
    
    def add_segment(self, position):
        new_segment = Turtle("square")
        new_segment.color("green")
        new_segment.penup()
        new_segment.goto(position)
        self.segments.append(new_segment)
    
    def extend(self):
        self.add_segment(self.segments[-1].position())

    def move(self):
        for seg_num in range(len(self.segments) - 1, 0, -1):
            new_x = self.segments[seg_num -1].xcor()
            new_y = self.segments[seg_num -1].ycor()
            self.segments[seg_num].goto(new_x, new_y)
        self.head.forward(MOVE_DISTANCE)

    def up(self):
        if self.head.heading() != DOWN:
            self.head.setheading(UP)
    
    def down(self):
        if self.head.heading() != UP:
            self.head.setheading(DOWN)
    
    def left(self):
        if self.head.heading() != RIGHT:
            self.head.setheading(LEFT)
    
    def right(self):
        if self.head.heading() != LEFT:
            self.head.setheading(RIGHT)
    
    def reset(self):
        for segment in self.segments:
            segment.hideturtle()
        self.segments.clear()
        self.create_snake()
        self.head = self.segments[0]

- The snake starts with three segments positioned in a line.
- The add_segment method adds new segments to the snake.
- The move method moves each segment to the position of the segment in front of it, creating a slithering effect.
- The direction methods (up, down, left, right) control the snake's direction and prevent it from reversing.

### Putting it all together

Finally, I set up the game itself. This script initializes everything, runs the game loop, and handles interactions between the snake, food, and scoreboard. I made the snake start slow and get faster with each food it eats. Plus, I added a countdown to build anticipation before the game starts.

In [6]:
# Setup screen
screen = Screen()
screen.setup(width=600, height=600)
screen.bgcolor("black")
screen.title("The Very Hungry Snake Game")
screen.tracer(0)

# Create snake, food, and scoreboard
snake = Snake()
food = Food(snake.segments)
scoreboard = Scoreboard()

# Listen to keyboard inputs
screen.listen()
screen.onkey(snake.up, "Up")
screen.onkey(snake.down, "Down")
screen.onkey(snake.left, "Left")
screen.onkey(snake.right, "Right")

# Global variables for game control
initial_speed = 0.2  # Slower initial speed
speed = initial_speed
game_is_on = False

# Countdown function using ontimer
def countdown():
    counter = Turtle()
    counter.hideturtle()
    counter.color("white")
    counter.penup()
    counter.goto(0, 0)

    def update_countdown(num):
        if num > 0:
            counter.clear()
            counter.write(f"{num}", align="center", font=("Arial", 48, "normal"))
            screen.ontimer(lambda: update_countdown(num - 1), 1000)
        else:
            counter.clear()
            counter.write("Go!", align="center", font=("Arial", 48, "normal"))
            screen.ontimer(lambda: clear_go_and_start_game(), 1000)  # Clear "Go!" and start game

    def clear_go_and_start_game():
        counter.clear()
        start_game_loop()
    
    update_countdown(3)

# Reset game state
def reset_game():
    global speed, game_is_on
    snake.reset()
    food.refresh()
    scoreboard.reset()
    speed = initial_speed
    game_is_on = True

# Prompt user to play again
def play_again_prompt():
    prompt = Turtle()
    prompt.hideturtle()
    prompt.color("white")
    prompt.penup()
    prompt.goto(0, -50)
    prompt.write("Press 'Y' to play again or 'N' to exit", align="center", font=("Arial", 24, "normal"))

    def play_again():
        prompt.clear()
        reset_game()
        countdown()

    def exit_game():
        screen.bye()

    screen.onkey(play_again, "y")
    screen.onkey(exit_game, "n")
    screen.listen()

# Main game loop
def start_game_loop():
    global game_is_on, speed
    game_is_on = True
    screen.tracer(0)  # Turn off automatic screen updates

    while game_is_on:
        screen.update()
        time.sleep(speed)
        snake.move()

        # Detect collision with food
        if snake.head.distance(food) < 15:
            food.refresh()
            snake.extend()
            scoreboard.increase_score()
            # Increase speed as the score increases
            speed = initial_speed * (0.9 ** scoreboard.score)
        
        # Detect collision with wall
        if snake.head.xcor() > 280 or snake.head.xcor() < -280 or snake.head.ycor() > 280 or snake.head.ycor() < -280:
            game_is_on = False
            scoreboard.game_over()
            play_again_prompt()
        
        # Detect collision with tail
        for segment in snake.segments[1:]:
            if snake.head.distance(segment) < 10:
                game_is_on = False
                scoreboard.game_over()
                play_again_prompt()

    screen.tracer(1)  # Turn on automatic screen updates

# Start the game for the first time
def start_game():
    countdown()

# Start the game
start_game()

screen.exitonclick()

- Set up the screen with a black background and a title.
- Created instances of Snake, Food, and Scoreboard.
- Set up keyboard controls to move the snake.
- Added a countdown function for a 3-2-1-Go! countdown before the game starts.
- The game starts slow with initial_speed and gets faster each time the snake eats.
- The main game loop updates the screen, moves the snake, and checks for collisions with the food, walls, and its own tail.
- Added a replay function